In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Feature Engineering
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Machine Learning
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, make_scorer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor

import optuna

# Combine Datasets

In [ ]:
train_density = pd.read_csv("/kaggle/input/buliding-density/training_data_with_density.csv")
train_density.head()

In [ ]:
train_satellite = pd.read_csv("/kaggle/input/satellite-data/training_data_with_satellite.csv")
train_satellite.head()

In [ ]:
train_satellite = train_satellite.drop(["Longitude","Latitude","datetime","UHI Index"], axis=1)
train_concat = pd.concat([train_density, train_satellite], axis = 1)
train_concat.head()

In [ ]:
validation_density = pd.read_csv("/kaggle/input/buliding-density/validation_data_with_density.csv")
validation_density.head()

In [ ]:
validation_satellite = pd.read_csv("/kaggle/input/satellite-data/validation_data_with_satellite.csv")
validation_satellite.head()

In [ ]:
validation_satellite = validation_satellite.drop(["Longitude","Latitude","UHI Index"], axis=1)
validation_concat = pd.concat([validation_density, validation_satellite], axis=1)
validation_concat.head()

# Select features 

In [ ]:
features = ['B01','B06','NDVI','NDBI','NDWI','LST','density']
train_df = train_concat[features + ["UHI Index"]]
train_df

# Remove duplicates from training data

In [ ]:
# Remove duplicate rows from the DataFrame based on specified columns and keep the first occurrence
for col in features:
    # Check if the value is a numpy array and has more than one dimension
    train_df[col] = train_df[col].apply(lambda x: tuple(x) if isinstance(x, np.ndarray) and x.ndim > 0 else x)

# Now remove duplicates
uhi_data = train_df.drop_duplicates(subset=features, keep='first')
uhi_data.head()

In [ ]:
uhi_data.shape

In [ ]:
# Resetting the index of the dataset
uhi_data=uhi_data.reset_index(drop=True)

In [ ]:
uhi_data.isna().sum()

# Model Building

In [ ]:
# Split the data into features (X) and target (y), and then into training and testing sets
X = uhi_data.drop(columns=['UHI Index']).values
y = uhi_data ['UHI Index'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

**Feature Scaling**

In [ ]:
# Scale the training and test data using standardscaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

**Model Training**

XGBoost

In [ ]:
# Define the hyperparameter grid for RandomizedSearchCV
param_grid = {
    'n_estimators': [200, 300, 500, 700, 900, 1000],          # Number of trees
    'max_depth': [None, 5, 10, 30, 50, 100],           # Maximum depth of trees
    'learning_rate': [0.01, 0.05, 0.1, 0.2],           # Step size shrinkage
    'subsample': [0.6, 0.8, 1.0],                      # Fraction of samples used per tree
    'colsample_bytree': [0.6, 0.8, 1.0],               # Fraction of features used per tree
    'min_child_weight': [1, 2, 5, 10],                 # Minimum sum of instance weight needed in a child
    'gamma': [0, 0.1, 0.2, 0.5]                        # Minimum loss reduction required for further partitioning
}

# Define the model
xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)

# Create the RandomizedSearchCV instance
random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_grid,
    n_iter=500,  # Number of random combinations to try
    cv=5,       # 3-fold cross-validation
    scoring=make_scorer(r2_score),
    n_jobs=-1,  # Use all available cores
    verbose=1,  # Print progress
    random_state=42
)

# Perform the search
random_search.fit(X_train, y_train)

# Retrieve the best parameters
best_params = random_search.best_params_
print("Best Hyperparameters found: ", best_params)

# Train the model with the best parameters
best_xgb = XGBRegressor(**best_params)
best_xgb.fit(X_train, y_train)

r2_xgb = r2_score(y_test, best_xgb.predict(X_test))

# Print model details
print(f"Best model has {best_xgb.n_estimators} trees and max depth of {best_xgb.max_depth}.")
print(f"R^2 score: {r2_xgb}")

Random Forest Regressor

In [ ]:
# Define the hyperparameter grid for RandomizedSearchCV
param_grid = {
    'n_estimators': [100, 200, 300, 500, 700, 900, 1000],          # Number of trees
    'max_depth': [None, 5, 10, 30, 50, 100],           # Maximum depth of trees
    'min_samples_split': [2, 5, 10],             # Minimum samples required to split a node
    'min_samples_leaf': [1, 2, 4],                # Minimum samples required in a leaf node
    'max_features': ['sqrt', 'log2', 0.1, 0.5],  # Number of features to consider for the best split
}


# Define the model
rf_model = RandomForestRegressor(random_state=42)

# Create the RandomizedSearchCV instance
random_search = RandomizedSearchCV(
    estimator=rf_model,
    param_distributions=param_grid,
    n_iter=500,               # Number of random combinations to try
    cv=5,                    # 3-fold cross-validation
    scoring=make_scorer(r2_score),
    n_jobs=-1,               # Use all available cores
    verbose=1,               # Print progress
    random_state=42          # Ensure reproducibility
)

# Perform the search
random_search.fit(X_train, y_train)

# Retrieve the best parameters
best_params = random_search.best_params_
print("Best Hyperparameters found: ", best_params)

# Train the model with the best parameters
best_rf = RandomForestRegressor(**best_params)
best_rf.fit(X_train, y_train)

r2_rf = r2_score(y_test, best_rf.predict(X_test))

# Print model details
print(f"Best model has {best_rf.n_estimators} trees and max depth of {best_rf.max_depth}.")
print(f"R^2 score: {r2_rf}")

Ensemble XGBoost and Random Forest

In [ ]:
# Function to compute the weighted ensemble predictions
def ensemble_predictions(weight, rf_preds, xgb_preds):
    return weight * xgb_preds + (1 - weight) * rf_preds

# Define the objective function for Optuna
def objective(trial):
    # Suggest a weight for the XGBoost model
    weight = trial.suggest_float('weight', 0.0, 1.0)
    
    # Generate predictions for both models
    rf_preds = best_rf.predict(X_train)
    xgb_preds = best_xgb.predict(X_train)
    
    # Combine predictions using the suggested weight
    ensemble_preds = ensemble_predictions(weight, rf_preds, xgb_preds)
    
    # Calculate R² score for the ensemble
    return r2_score(y_train, ensemble_preds)

# Train the models on the training set
best_rf.fit(X_train, y_train)
best_xgb.fit(X_train, y_train)

# Create an Optuna study to find the best weight
study = optuna.create_study(direction='maximize', study_name="Ensemble Optimization")
study.optimize(objective, n_trials=10, show_progress_bar=True)

# Retrieve the best weight
best_weight = study.best_params['weight']
print(f"Best weight for XGBoost: {best_weight:.4f}")
print(f"Best weight for Random Forest: {1- best_weight:.4f}")

In [ ]:
# Save the final ensemble model as a function
def ensembled_model(X):
    rf_preds = best_rf.predict(X)
    xgb_preds = best_xgb.predict(X)
    return ensemble_predictions(best_weight, rf_preds, xgb_preds)

**Model Evaluation**

In [ ]:
# Evaluate the ensemble on the test set
final_r2 = r2_score(y_test, ensembled_model(X_test))
print(f"Final R² score of the ensemble: {final_r2:.4f}")

# Submission

In [ ]:
validation_concat.head()

In [ ]:
submission_val_data = validation_concat[features]
submission_val_data.head()

In [ ]:
# Feature Scaling 
submission_val_data = submission_val_data.values
transformed_submission_data = sc.transform(submission_val_data)

In [ ]:
#Making predictions
final_predictions = ensembled_model(transformed_submission_data)
final_prediction_series = pd.Series(final_predictions)

In [ ]:
#Combining the results into dataframe
submission_df = pd.DataFrame({'Longitude':validation_concat['Longitude'].values, 'Latitude':validation_concat['Latitude'].values, 'UHI Index':final_prediction_series.values})

In [ ]:
#Displaying the sample submission dataframe
submission_df.head()

In [ ]:
#Dumping the predictions into a csv file.
submission_df.to_csv("submission.csv",index = False)